In [ ]:
import pandas as pd

In [ ]:
company_name: str = '万华化学（福建）有限公司'
project_code: str = '23ZKP0019'

file_path: str = 'WT23ZKP0019系统生成编号.xlsx'

In [ ]:
available_cols: list[str] = [
    '样品类型',
    '样品编号',
    '样品名称',
    '检测参数',
    '单元',
    '工种/岗位',
    '检测地点',
    '测点编号',
    '第几天',
    '第几个频次',
    '采样方式',
    '作业人数',
    '日接触时长/h',
    '周工作天数/d',
]

df = pd.read_excel(file_path, sheet_name=0, usecols=available_cols)
df['样品编号'] = df['样品编号'].apply(lambda x: x.replace(project_code, ''))

df.head()

In [ ]:
raw_blank_df = df.query('样品类型 == "空白样"').reset_index(drop=True)

raw_blank_df.head()

In [ ]:
blank_df = raw_blank_df[raw_blank_df['第几天'] == 1].pivot(
    index='检测参数',
    columns='第几个频次',
    values='样品编号'
).rename(columns={1: '空白编号1', 2: '空白编号2'})

blank_df

In [ ]:
raw_point_df = df.query('样品类型 == "普通样" and 采样方式 == "定点" and 样品名称 != "工作场所物理因素"').reset_index(drop=True)
raw_point_df['样品编号'] = raw_point_df['样品编号'].astype(int)

raw_point_df.head()

In [ ]:
groupby_point_df = raw_point_df.groupby(
    [
        '测点编号',
        '单元',
        '检测地点',
        '工种/岗位',
        '检测参数',
        '第几天',
        '日接触时长/h'
    ]
)['样品编号'].agg(list).reset_index(drop=False)
groupby_point_df['样品数量'] = groupby_point_df['样品编号'].apply(lambda x: len(x))
groupby_point_df['是否合并代表时长'] = groupby_point_df.apply(lambda df: True if df['日接触时长/h'] / df['样品数量'] < 0.25 else False, axis=1)

groupby_point_df.head()

In [ ]:
pivoted_point_df = pd.pivot_table(
    data=raw_point_df[raw_point_df['第几天'] == 1],
    index=[
        '测点编号',
        '单元',
        '检测地点',
        '工种/岗位',
        '日接触时长/h',
        '检测参数'
    ],
    # aggfunc={'样品编号': min, '样品编号': max}
    values='样品编号',
    aggfunc=[min, max]
).rename(columns={'min': '起始样品编号', 'max': '终止样品编号'}).reset_index().droplevel(1, axis=1)

pivoted_point_df['样品数量'] = pivoted_point_df['终止样品编号'] - pivoted_point_df['起始样品编号'] + 1

point_df = pivoted_point_df.merge(
    blank_df,
    how='outer',
    left_on='检测参数',
    right_on='检测参数'
).fillna({'空白编号1': '', '空白编号2': ''})

point_df['是否合并代表时长'] = point_df.apply(lambda df: True if df['日接触时长/h'] / df['样品数量'] < 0.25 else False, axis=1)

point_df.head()